In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.xls')
data.info()
data.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
951,tt0157472,26000000,38793283,Clockstoppers,Jesse Bradford|Paula GarcÃ©s|Robin Thomas|Fren...,Jonathan Frakes,"The adventure of a lifetime, in a few mere sec...","Until now, Zak Gibbs' greatest challenge has b...",94,Adventure|Family|Science Fiction|Thriller,Paramount Pictures|Nickelodeon Movies|Valhalla...,3/17/2002,4.5,2002
1766,tt0498399,21000000,54926886,We Own the Night,Joaquin Phoenix|Eva Mendes|Alex Veadov|Mark Wa...,James Gray,Two brothers on opposite sides of the law. Bey...,A New York nightclub manager tries to save his...,117,Drama|Crime|Thriller,Columbia Pictures|Industry Entertainment|2929 ...,5/25/2007,6.4,2007
677,tt0482606,9000000,82391145,The Strangers,Liv Tyler|Scott Speedman|Alex Fisher|Peter Cla...,Bryan Bertino,Lock the door. Pretend you're safe.,"After returning from a wedding reception, a co...",90,Thriller|Horror|Mystery,Rogue Pictures|Vertigo Entertainment,5/29/2008,5.9,2008
1059,tt0763831,40000000,22044277,A Thousand Words,Eddie Murphy|Allison Janney|Cliff Curtis|Clark...,Brian Robbins,He only has 1000 words left to discover what m...,"Jack McCall is a fast-talking literary agent, ...",91,Drama|Comedy,Paramount Pictures|DreamWorks SKG|Saturn Films...,3/7/2012,5.8,2012
907,tt0160184,55000000,6416302,D-Tox,Sylvester Stallone|Courtney B. Vance|Tom Beren...,Jim Gillespie,Survival is a Killer.,A disgraced FBI agent with a drinking problem ...,96,Action|Thriller,Universal Pictures,1/4/2002,5.8,2002


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# Добавим столбец profit к имеющемуся DataFrame
profit = pd.DataFrame(list(list(data.revenue - data.budget)), columns=['profit'])
data = pd.concat([data, profit], axis=1)


In [5]:
# Переведем release_time из строкового типа в time.struct_time
import time

dt = pd.DataFrame()
for i in list(data.release_date):
    old_str = i.split('/')
    if 1 <= int(old_str[0]) <= 9:
        new_str = '0' + old_str[0] + '/'
    else:
        new_str = old_str[0] + '/'
    if 1 <= int(old_str[1]) <= 9:
        new_str += '0' + old_str[1] + '/'
    else:
        new_str += old_str[1] + '/'
    new_str += old_str[2][2:]
    dt = dt.append({'release_date': time.strptime(new_str, "%m/%d/%y")[:3]}, ignore_index=True)

data = data.drop(columns='release_date')
data = pd.concat([data, dt], axis=1)


In [6]:
# Дополним исходную таблицу столбцами жанров, где значением является 1 или 0
genres = {}
k = 0

for i in data.genres:
    old_str = i.split('|')
    for j in old_str:
        genres[j] = []

genres_df = pd.DataFrame(genres)
        
for i in data.genres:
    old_str = i.split('|')
    for j in old_str:
        genres_df.loc[k, j] = 1
    k += 1

genres_df = genres_df.fillna(0)
data = data.drop(['genres'], axis=1)
data = pd.concat([data, genres_df], axis=1)
display(data.sample(5))
display(genres_df.sample(5))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,production_companies,...,Animation,Comedy,Mystery,Romance,War,History,Music,Horror,Documentary,Foreign
1302,tt0884726,70000000,18662027,Legends of Oz: Dorothy's Return,Lea Michele|Dan Aykroyd|Patrick Stewart|Hugh D...,Dan St. Pierre,There's trouble in OZ...,"Dorothy wakes up in post-tornado Kansas, only ...",88,Prana Animation Studios|Summertime Entertainment,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1844,tt0190138,41300000,106371651,The Whole Nine Yards,Bruce Willis|Matthew Perry|Rosanna Arquette|Mi...,Jonathan Lynn,"In the heart of suburbia, a hit man with heart...",A mobster named Jimmy the Tulip agrees to coop...,98,Franchise Pictures|Warner Bros.|Morgan Creek P...,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,tt4178092,5000000,58978653,The Gift,Jason Bateman|Rebecca Hall|Joel Edgerton|David...,Joel Edgerton,Not every gift is welcome.,A husband and wife try to reinvigorate their r...,108,Blumhouse Productions|Blue-Tongue Films|STX En...,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
565,tt0236784,21000000,28008462,The Tailor of Panama,Pierce Brosnan|Geoffrey Rush|Jamie Lee Curtis|...,John Boorman,"In a place this treacherous, what a good spy n...",A British spy is banished to Panama after havi...,109,Columbia Pictures|Merlin Films,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
616,tt0493464,75000000,258270008,Wanted,Angelina Jolie|James McAvoy|Morgan Freeman|Tho...,Timur Bekmambetov,Choose your destiny.,Doormat Wesley Gibson discovers that his recen...,110,Universal Pictures|Spyglass Entertainment|Baze...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Action,Adventure,Science Fiction,Thriller,Fantasy,Crime,Western,Drama,Family,Animation,Comedy,Mystery,Romance,War,History,Music,Horror,Documentary,Foreign
1200,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
745,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1303,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
421,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [7]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# "+"

In [8]:
display(data[data.budget == data.budget.max()].original_title)

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
answers['2'] = 'Gods and Generals (tt0279111)'
# "+"

In [10]:
display(data[data.runtime == data.runtime.max()].original_title)

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?

In [11]:
answers['3'] = 'Winnie the Pooh (tt1449283)'
# "+"

In [12]:
display(data[data.runtime == data.runtime.min()].original_title)

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [13]:
answers['4'] = '110'
# "+"

In [14]:
display(data.runtime.mean())

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [15]:
answers['5'] = '107'
# "+"

In [16]:
display(data.runtime.median())

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [17]:
answers['6'] = 'Avatar (tt0499549)'
# "+"

In [18]:
display(data[data.profit == data.profit.max()].original_title)

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [19]:
answers['7'] = 'The Lone Ranger (tt1210819)'
# "+"

In [20]:
display(data[data.profit == data.profit.min()].original_title)

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [21]:
answers['8'] = '1478'
# "+"

In [22]:
display(len(data[data.revenue > data.budget]))

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
answers['9'] = 'The Dark Knight (tt0468569)'
# "+"

In [24]:
df = data[data.release_year == 2008]
display(df[df.revenue == df.revenue.max()].original_title)

599    The Dark Knight
Name: original_title, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [25]:
answers['10'] = 'The Lone Ranger (tt1210819)'
# "+"

In [26]:
df = data[(2012 <= data.release_year) & (data.release_year <= 2014)]
display(df[df.profit == df.profit.min()].original_title)

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [27]:
answers['11'] = 'Drama'
# "+"

In [28]:
display(genres_df.sum() == genres_df.sum().max())

Action             False
Adventure          False
Science Fiction    False
Thriller           False
Fantasy            False
Crime              False
Western            False
Drama               True
Family             False
Animation          False
Comedy             False
Mystery            False
Romance            False
War                False
History            False
Music              False
Horror             False
Documentary        False
Foreign            False
dtype: bool

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [29]:
answers['12'] = 'Drama'
# "+"

In [30]:
g_df1 = pd.concat([data.profit, genres_df], axis=1)
g_df1 = g_df1.fillna(0)
g_df1 = g_df1[g_df1 > 0]
g_df1 = g_df1.drop(['profit'], axis=1)
g_df1 = g_df1.fillna(0)
display(g_df1.sum() == g_df1.sum().max())

Action             False
Adventure          False
Science Fiction    False
Thriller           False
Fantasy            False
Crime              False
Western            False
Drama               True
Family             False
Animation          False
Comedy             False
Mystery            False
Romance            False
War                False
History            False
Music              False
Horror             False
Documentary        False
Foreign            False
dtype: bool

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [31]:
answers['13'] = 'Peter Jackson'
# "+"

In [32]:
df = data.groupby(['director'])['revenue'].sum()
display(df[df == df.max()])

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [63]:
answers['14'] = 'Robert Rodriguez'
# "+"

In [34]:
directors = {}

for i in range(len(data)):
    for j in data.loc[i].director.split('|'):
        if data.loc[i].Action == 1:
            directors[j] = 0

for i in range(len(data)):
    for j in data.loc[i].director.split('|'):
        if data.loc[i].Action == 1:
            directors[j] += 1

for key, value in directors.items():
    if value == max(directors.values()):
        display(key)
    

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [35]:
answers['15'] = 'Chris Hemsworth'
# "+"

In [67]:
actors = {}
for i in data.cast:
    for j in i.split('|'):
        actors.setdefault(j, 0)
        
df = data[data.release_year == 2012]
for i in list(df.index):
    for j in df.loc[i].cast.split('|'):
        actors[j] += df.loc[i].revenue

for key, value in actors.items():
    if value == max(actors.values()):
        display(key)

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [37]:
answers['16'] = 'Matt Damon'
# "+"

In [38]:
df = data[data.budget > 50000000]

actors = {}
for i in df.cast:
    for j in i.split('|') :
        actors.setdefault(j, 0)

for i in list(df.index):
    for j in df.loc[i].cast.split('|'):
        actors[j] += 1

for key, value in actors.items():
    if value == max(actors.values()):
        display(key)

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [39]:
answers['17'] = 'Action'
# "+"

In [40]:
for i in genres:
    genres[i] = 0

for i in range(len(data)):
    for j in data.loc[i].cast.split('|'):
        if j == 'Nicolas Cage':
            for k in list(genres_df[genres_df.index == i].columns):
                if genres_df[genres_df.index == i][k].values[0] == 1:
                    genres[k] += 1

for key, value in genres.items():
    if value == max(genres.values()):
        display(key)

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [41]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'
# "+"

In [42]:
df = pd.DataFrame()
for i in range(len(data)):
    for j in data.loc[i].production_companies.split('|'):
        if j == 'Paramount Pictures':
            df = df.append(data[data.index == i], ignore_index=True)
display(df[df.profit == df.profit.min()].original_title)

54    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [43]:
answers['19'] = '2015'
# "+"

In [44]:
df = data.groupby(['release_year'])['revenue'].sum()
display(df[df == df.max()])

release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [45]:
answers['20'] = '2014'
# "+"

In [46]:
df = pd.DataFrame()
for i in range(len(data)):
    for j in data.loc[i].production_companies.split('|'):
        if 'Warner Bros' in j:
            df = df.append(data[data.index == i], ignore_index=True)

df1 = df.groupby(['release_year'])['revenue'].sum()
display(df1[df1 == df1.max()])

release_year
2014    3246549646
Name: revenue, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [47]:
answers['21'] = 'Сентябрь'
# "+"

In [48]:
months = {str(i): 0 for i in range(1, 13)}
for i in range(len(data)):
    months[str(data.release_date.loc[i][1])] += 1
display(max(months))

'9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [49]:
answers['22'] = '450'
# "+"

In [50]:
months = {str(i): 0 for i in range(1, 13)}
for i in range(len(data)):
    months[str(data.release_date.loc[i][1])] += 1
display(months['6'] + months['7'] + months['8'])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [51]:
answers['23'] = 'Peter Jackson'
# "+"

In [52]:
directors = {}

for i in range(len(data)):
    for j in data.loc[i].director.split('|'):
        directors[j] = 0

for i in range(len(data)):
    for j in data.loc[i].director.split('|'):
        if data.release_date.loc[i][1] == 12 or data.release_date.loc[i][1] == 1 or data.release_date.loc[i][1] == 2:
            directors[j] += 1

for key, value in directors.items():
    if value == max(directors.values()):
        display(key)

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [64]:
answers['24'] = 'Four By Two Productions'
# "+"

In [54]:
names = {}

for i in range(len(data)):
    for j in data.loc[i].production_companies.split('|'):
        names[j] = 0

for i in range(len(data)):
    for j in data.loc[i].production_companies.split('|'):
        if len(data.loc[i].original_title) > names[j]:
            names[j] = len(data.loc[i].original_title)

for key, value in names.items():
    if value == max(names.values()):
        display(key)

'Twentieth Century Fox Film Corporation'

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [55]:
answers['25'] = 'Midnight Picture Show'
# "+"

In [56]:
names = {}

for i in range(len(data)):
    for j in data.loc[i].production_companies.split('|'):
        names[j] = [0]*2

for i in range(len(data)):
    for j in data.loc[i].production_companies.split('|'):
        names[j][0] += len(data.loc[i].overview)
        names[j][1] += 1

for i in names:
    names[i] = names[i][0] / names[i][1]

for key, value in names.items():
    if value == max(names.values()):
        display(key)

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу?

In [57]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
# "+"

In [58]:
df = data[['original_title', 'vote_average']].sort_values(['vote_average'], ascending=False)
df.index = list(df.original_title)
df = df['vote_average']
display(df.head(len(df)//100))

The Dark Knight                                  8.1
Interstellar                                     8.0
The Imitation Game                               8.0
Inside Out                                       8.0
Room                                             8.0
The Wolf of Wall Street                          7.9
Gone Girl                                        7.9
12 Years a Slave                                 7.9
Guardians of the Galaxy                          7.9
The Lord of the Rings: The Return of the King    7.9
Memento                                          7.9
Inception                                        7.9
The Pianist                                      7.9
The Grand Budapest Hotel                         7.9
Her                                              7.8
Spotlight                                        7.8
Big Hero 6                                       7.8
The Fault in Our Stars                           7.8
Name: vote_average, dtype: float64

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [59]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
# "+"

In [60]:
actors = {}

for i in data.cast:
    for j in i.split('|'):
        actors.setdefault(j, 0)
        
actors = pd.DataFrame(actors, index=list(actors.keys()))

for i in range(len(data)):
    for j in data.loc[i].cast.split('|'):
        for k in data.loc[i].cast.split('|'):
            if j != k:
                actors.loc[j].loc[k] += 1
                actors.loc[k].loc[j] += 1
                
display(actors[actors.max() == actors.max().max()][actors[actors.max() == actors.max().max()].index])

,Daniel Radcliffe,Emma Watson,Rupert Grint
Daniel Radcliffe,0,16,16
Emma Watson,16,0,16
Rupert Grint,16,16,0


# Submission

In [65]:
display(answers)

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [66]:
display(len(answers))

27